#Group SOMETHING 1D ML project
1. Loy Pek Yong 1004475


#Part 1

In [30]:
# import libraries
import pandas as pd
import numpy as np

# read data
ES_train = pd.read_csv("./ES/train", sep=" ", header=None, names=["word", "label"], encoding="utf-8", quoting=3)
ES_dev_in = pd.read_csv("./ES/dev.in", sep=" ", header=None, names=["word"], encoding="utf-8", quoting=3)
ES_dev_out = pd.read_csv("./ES/dev.out", sep=" ", header=None, names=["label"], encoding="utf-8", quoting=3)

RU_train = pd.read_csv("./RU/train", sep=" ", header=None, names=["word", "label"], encoding="utf-8", quoting=3)
RU_dev_in = pd.read_csv("./RU/dev.in", sep=" ", header=None, names=["word"], encoding="utf-8", quoting=3)
RU_dev_out = pd.read_csv("./RU/dev.out", sep=" ", header=None, names=["label"], encoding="utf-8", quoting=3)



In [31]:
# Part 1 Point 1
# function that returns the emission parameters with input of training set

def train_emission_param(training_set):
# Count the number of occurrences of each label and each (label, observed value) pair
    label_counts, emission_counts = {}, {}

    for observation, label in training_set:
        if label not in label_counts:
            label_counts[label] = 0
        label_counts[label] += 1
        if (label, observation) not in emission_counts:
            emission_counts[(label, observation)] = 0
        emission_counts[(label, observation)] += 1

# Estimate the emission parameters from the counts
    emission_param = {}
    for label, observation in emission_counts:
        emission_param[(label, observation)] = emission_counts[(label, observation)] / label_counts[label]
    return emission_param, emission_counts, label_counts


In [32]:
# Part 1 Point 2
# function that returns the test emission parameters with input of test set
def test_emission_param(test_set, emission_param, emission_counts, label_counts):
    # predict the label of the test set
    k = 1
    special_label = '#UNK#'
    
    for observation, label in test_set:

        if (label, observation) not in emission_param:
            # k/(count(y)+k)
            emission_param[(special_label, observation)] = k/(label_counts[label]+k)
            k += 1
        
        else:
            #count(y-> x)/(count(y)+k)
            emission_param[(label, observation)] = (emission_counts[(label, observation)]+k)/(label_counts[label]+k)

    return emission_param

In [40]:
# Part 1 Point 3

def sentiment_analysis(training_set, test_set):

    special_label = '#UNK#' # special label for unknown words
    # train the emission parameters
    train_emission_param_data, emission_counts, label_counts = train_emission_param(training_set)
    # test the emission parameters
    test_emission_param_data = test_emission_param(test_set, train_emission_param_data, emission_counts, label_counts)

    # predict the label of the test set
    predicted = {}
    for observation, label in test_set:
        
        # if the observation is not in the emission parameters, assign the special label
        if (special_label, observation) in test_emission_param_data:
            predicted[observation] = special_label
        else:
            # assign the label with the highest emission parameter
            predicted[observation] = max(test_emission_param_data, key=test_emission_param_data.get)[0]

def precision(predicted, test_set):
    # calculate the precision
    correct = 0
    total = 0
    if predicted:
        for observation, label in test_set:
            if observation in predicted and predicted[observation] == label:
                correct += 1
            total += 1
    return correct/total if total > 0 else 0

def recall(predicted, compare_set):
    # calculate the recall
    correct = 0
    total = 0
    if predicted:
        for observation, label in compare_set:
            if observation in predicted and predicted[observation] == label :
                correct += 1
            total += 1
    return correct/total if total > 0 else 0

def f1_score(precision, recall):
    # calculate the f1 score
    return 2*precision*recall/(precision+recall)


ZeroDivisionError: division by zero